# Use AutoAI and batch deployment to predict credit risk with Watson Machine Learning REST API

This notebook contains steps and code to demonstrate support of AutoAI experiments in Watson Machine Learning Service. It introduces commands for getting data, training experiments, persisting pipelines, publishing models, deploying models and scoring.

Some familiarity with cURL is helpful. This notebook uses cURL examples.


## Learning goals

The learning goals of this notebook are:

-  Working with Watson Machine Learning experiments to train AutoAI models.
-  Downloading computed models to local storage.
-  Batch deployment and scoring of trained model.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)  
2.	[Experiment definition](#experiment_definition)  
3.	[Experiment Run](#run)  
4.	[Historical runs](#runs)  
5.	[Deploy and Score](#deploy_and_score)  
6.	[Cleaning](#cleaning) 
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `password`.

In [200]:
%env USERNAME=
%env PASSWORD=
%env DATAPLATFORM_URL=

%env SPACE_ID=

env: USERNAME=
env: PASSWORD=
env: DATAPLATFORM_URL=
env: SPACE_ID=


<a id="wml_token"></a>
### Getting WML authorization token for further cURL calls

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-token" target="_blank" rel="noopener no referrer">Example of cURL call to get WML token</a>

In [2]:
%%bash --out token

curl -sk -X GET \
    --user $USERNAME:$PASSWORD \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/v1/preauth/validateAuth" \
    | cut -d '"' -f 44

In [199]:
%env TOKEN=$token 

<a id="space_creation"></a>
### Space creation
**Tip:** If you do not have `space` already created, please convert below three cells to `code` and run them.

First of all, you need to create a `space` that will be used in all of your further cURL calls. 
If you do not have `space` already created, below is the cURL call to create one.

<a href="https://cpd-spaces-api.eu-gb.cf.appdomain.cloud/#/Spaces/spaces_create" 
target="_blank" rel="noopener no referrer">Space creation</a>

Space creation is asynchronous. This means that you need to check space creation status after creation call.
Make sure that your newly created space is `active`.

<a href="https://cpd-spaces-api.eu-gb.cf.appdomain.cloud/#/Spaces/spaces_get" 
target="_blank" rel="noopener no referrer">Get space information</a>

<a id="experiment_definition"></a>
## 2. Experiment / optimizer configuration

Provide input information for AutoAI experiment / optimizer:
- `name` - experiment name
- `learning_type` - type of the problem
- `label` - target column name
- `scorer_for_ranking` - optimization metric
- `holdout_param` - percentage of training data to use as a holdout [0 - 1]
- `daub_include_only_estimators` - list of estimators to use

You can modify `parameters` section of the following cURL call to change AutoAI experiment settings.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Pipelines/pipelines_create" 
target="_blank" rel="noopener no referrer">Define AutoAI experiment.</a>

In [4]:
%%bash --out pipeline_payload

PIPELINE_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "name": "Credit Risk Prediction - AutoAI", "description": "", "document": {"doc_type": "pipeline", "version": "2.0", "pipelines": [{"id": "autoai", "runtime_ref": "hybrid", "nodes": [{"id": "automl", "type": "execution_node", "parameters": {"stage_flag": true, "output_logs": true, "input_file_separator": ",", "optimization": {"learning_type": "binary", "label": "Risk", "max_num_daub_ensembles": 1, "daub_include_only_estimators": ["ExtraTreesClassifierEstimator", "GradientBoostingClassifierEstimator", "LGBMClassifierEstimator", "LogisticRegressionEstimator", "RandomForestClassifierEstimator", "XGBClassifierEstimator", "DecisionTreeClassifierEstimator"], "scorer_for_ranking": "roc_auc", "compute_pipeline_notebooks_flag": true, "run_cognito_flag": true, "holdout_param": 0.1}}, "runtime_ref": "autoai", "op": "kube"}]}], "runtimes": [{"id": "autoai", "name": "auto_ai.kb", "app_data": {"wml_data": {"hardware_spec": { "name": "M"}}}, "version": "3.0.2"}],"primary_pipeline": "autoai"}}'
echo $PIPELINE_PAYLOAD | python -m json.tool

In [5]:
%env PIPELINE_PAYLOAD=$pipeline_payload

env: PIPELINE_PAYLOAD={
    "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
    "name": "Credit Risk Prediction - AutoAI",
    "description": "",
    "document": {
        "doc_type": "pipeline",
        "version": "2.0",
        "pipelines": [
            {
                "id": "autoai",
                "runtime_ref": "hybrid",
                "nodes": [
                    {
                        "id": "automl",
                        "type": "execution_node",
                        "parameters": {
                            "stage_flag": true,
                            "output_logs": true,
                            "input_file_separator": ",",
                            "optimization": {
                                "learning_type": "binary",
                                "label": "Risk",
                                "max_num_daub_ensembles": 1,
                                "daub_include_only_estimators": [
                                    "ExtraTreesCla

In [6]:
%%bash --out pipeline_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$PIPELINE_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/pipelines?version=2020-08-01" \
    | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 5p

In [7]:
%env PIPELINE_ID=$pipeline_id

env: PIPELINE_ID=a8cbf66e-2e7d-4ec4-b81c-0c24b38ef35f


<a id="experiment_details"></a>
### Get experiment details
To retrieve AutoAI experiment / optimizer configuration you can follow below cURL GET call.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Pipelines/pipelines_get" 
target="_blank" rel="noopener no referrer">Get experiment / optimizer information</a>

In [8]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/pipelines/$PIPELINE_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "document": {
            "doc_type": "pipeline",
            "pipelines": [
                {
                    "id": "autoai",
                    "nodes": [
                        {
                            "id": "automl",
                            "op": "kube",
                            "parameters": {
                                "input_file_separator": ",",
                                "optimization": {
                                    "compute_pipeline_notebooks_flag": true,
                                    "daub_include_only_estimators": [
                                        "ExtraTreesClassifierEstimator",
                                        "GradientBoostingClassifierEstimator",
                                        "LGBMClassifierEstimator",
                                        "LogisticRegressionEstimator",
                                        "RandomForestClassifierEstimator",
                                 

<a id="training_connection"></a>
### Training data connection

Define connection information to COS bucket and training data CSV file. This example uses the German Credit Risk dataset. 

The dataset can be downloaded from [here](https://github.com/IBM/watson-machine-learning-samples/raw/master/data/credit_risk/credit_risk_training_light.csv ). You can also download it to local filesystem by running the cell below.

**Action**: Upload training data to COS bucket and enter location information in the next cURL examples.

In [9]:
%%bash

wget https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd3.5/data/credit_risk/credit_risk_training_light.csv \
     -O credit_risk_training_light.csv

--2020-10-01 08:40:42--  https://github.com/IBM/watson-machine-learning-samples/raw/master/data/credit_risk/credit_risk_training_light.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/data/credit_risk/credit_risk_training_light.csv [following]
--2020-10-01 08:40:43--  https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/data/credit_risk/credit_risk_training_light.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34787 (34K) [text/plain]
Saving to: ‘credit_risk_training_light.csv’

     0K .......... .......... .......... ...                  100%  925K=0,04s

2020-

<a id="cos_upload"></a>
### Create Data Asset
Upload your local dataset into Data Asset Storage

<a href="https://cloud.ibm.com/apidocs/watson-data-api#createdataassetv2" 
target="_blank" rel="noopener no referrer">Upload file as Data Asset</a>

In [10]:
%%bash --out data_asset_metadata

DATA_ASSET_METADATA='{"metadata": {"name": "autoai_training_data","description": "desc","asset_type": "data_asset","origin_country": "us","asset_category": "USER"},"entity": {"data_asset": {"mime_type": "text/csv"}}}'
echo $DATA_ASSET_METADATA | python -m json.tool

In [11]:
%env DATA_ASSET_METADATA=$data_asset_metadata

env: DATA_ASSET_METADATA={
    "metadata": {
        "name": "autoai_training_data",
        "description": "desc",
        "asset_type": "data_asset",
        "origin_country": "us",
        "asset_category": "USER"
    },
    "entity": {
        "data_asset": {
            "mime_type": "text/csv"
        }
    }
}


In [12]:
%%bash --out asset_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --data "$DATA_ASSET_METADATA" \
    "$DATAPLATFORM_URL/v2/assets?space_id=$SPACE_ID&version=2020-08-01" \
    | cut -d '"' -f 108

In [13]:
%env ASSET_ID=$asset_id

env: ASSET_ID=f5531b4b-d1e8-44e7-a464-67164a2b8a6e


In [14]:
%%bash --out attachment

ATTACHMENT='{"asset_type": "data_asset", "name": "credit_risk_training_light.csv", "mime": "text/csv"}'
echo $ATTACHMENT | python -m json.tool

In [15]:
%env ATTACHMENT=$attachment

env: ATTACHMENT={
    "asset_type": "data_asset",
    "name": "credit_risk_training_light.csv",
    "mime": "text/csv"
}


In [16]:
%%bash --out attachment_response

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --data "$ATTACHMENT" \
    "$DATAPLATFORM_URL/v2/assets/$ASSET_ID/attachments?space_id=$SPACE_ID&version=2020-08-01"


In [17]:
%env ATTACHMENT_RESPONSE=$attachment_response

env: ATTACHMENT_RESPONSE={"attachment_id":"7e81043b-134a-44c3-bb1c-f63628da2447","upload_id":"7be9c4bc-2091-4c75-8f93-d2fe00b115e1","data_partitions":1,"start_part_num":1,"url1":"/v2/asset_files/12667e3f-da77-4e8a-972a-eaf8ad3996e9/f5531b4b-d1e8-44e7-a464-67164a2b8a6e/7e81043b-134a-44c3-bb1c-f63628da2447?space_id=2219f598-6194-4e9a-9287-0730c69b5f4a&signature=lxGFHZYVh77a1OhGYEV6tQ%3D%3D%3ARPIz1KeMVaGUXIbjaj3CkDoDP%2BFnBh0iN9rYPNng7%2FIADu6QtGG4DS49CyXTHbLhCcNojg%2BbNt0cBp6L4foWvGlygXqBElvLpPytVOc0BNn05fXWEdSmIWKm9JbwcS1DjHlK51KJQVa2FClKNeXngoa5y3RQE0dUd6CJgerAUyhl2iBAEVBcLbVmTFg%2BRtwAPvbQZqJbyb24DW4U%2FT1n4syyFUnvk0dwdmwXG%2FA2OgsEJNnxd6aJKGjLrwJ%2FYbvge27eXIClzyPEo33dW5HSf9iEMumt7rP0Mn5iAitcheo44w1wzGGn%2BqI%3D","datasource_type":"81bafdbd-b7c6-45c5-a4fd-6ec135f66f4e","asset_type":"data_asset","attachment_type":"local","is_partitioned":false,"name":"credit_risk_training_light.csv","mime":"text/csv","user_data":{},"href":"/v2/assets/f5531b4b-d1e8-44e7-a464-67164a2b8a6e/attachments/7e

In [18]:
%%bash --out attachment_id

echo $ATTACHMENT_RESPONSE | cut -d '"' -f 4 | tr -d '\n'

In [19]:
%env ATTACHMENT_ID=$attachment_id

env: ATTACHMENT_ID=7e81043b-134a-44c3-bb1c-f63628da2447


In [20]:
%%bash --out attachment_url

echo $ATTACHMENT_RESPONSE | cut -d '"' -f 16 | tr -d '\n'

In [21]:
%env ATTACHMENT_URL=$attachment_url

env: ATTACHMENT_URL=/v2/asset_files/12667e3f-da77-4e8a-972a-eaf8ad3996e9/f5531b4b-d1e8-44e7-a464-67164a2b8a6e/7e81043b-134a-44c3-bb1c-f63628da2447?space_id=2219f598-6194-4e9a-9287-0730c69b5f4a&signature=lxGFHZYVh77a1OhGYEV6tQ%3D%3D%3ARPIz1KeMVaGUXIbjaj3CkDoDP%2BFnBh0iN9rYPNng7%2FIADu6QtGG4DS49CyXTHbLhCcNojg%2BbNt0cBp6L4foWvGlygXqBElvLpPytVOc0BNn05fXWEdSmIWKm9JbwcS1DjHlK51KJQVa2FClKNeXngoa5y3RQE0dUd6CJgerAUyhl2iBAEVBcLbVmTFg%2BRtwAPvbQZqJbyb24DW4U%2FT1n4syyFUnvk0dwdmwXG%2FA2OgsEJNnxd6aJKGjLrwJ%2FYbvge27eXIClzyPEo33dW5HSf9iEMumt7rP0Mn5iAitcheo44w1wzGGn%2BqI%3D


In [22]:
%%bash

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: multipart/form-data" \
    -F 'file=@credit_risk_training_light.csv' \
    "$DATAPLATFORM_URL$ATTACHMENT_URL"
    

{"status":"Asset created: The asset was successfully uploaded."}

The response should looks like this:  
    `{"status":"Asset created: The asset was successfully uploaded."}`

In [23]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$DATAPLATFORM_URL/v2/assets/$ASSET_ID/attachments/$ATTACHMENT_ID/complete?space_id=$SPACE_ID&version=2020-08-01"
    

{"attachment_id":"7e81043b-134a-44c3-bb1c-f63628da2447","asset_type":"data_asset","name":"credit_risk_training_light.csv","mime":"text/csv","size":34787}

<a id="run"></a>
## 3. Experiment run

This section provides samples about how to trigger AutoAI experiment via cURL calls.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_create" 
target="_blank" rel="noopener no referrer">Schedule a training job for AutoAI experiment</a>

In [54]:
%%bash --out random_id

openssl rand -hex 20

In [55]:
%env RANDOM_ID=$random_id

env: RANDOM_ID=eab90a7e0841c00db04266373c27ef71a0f312b5


In [56]:
%%bash --out training_payload

TRAINING_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "training_data_references": [{"type": "data_asset", "id": "credit_risk_training_light.csv", "connection": {}, "location": {"href": "/v2/assets/'"$ASSET_ID"'?space_id='"$SPACE_ID"'"}}], "results_reference": {"type": "fs", "id": "autoai_results", "connection": {}, "location": {"path": "/spaces/'$SPACE_ID'/assets/auto_ml/auto_ml_curl.'$RANDOM_ID'/wml_data"}}, "tags": [{"value": "autoai"}], "pipeline": {"href": "/v4/pipelines/'"$PIPELINE_ID"'"}}'
echo $TRAINING_PAYLOAD | python -m json.tool

In [57]:
%env TRAINING_PAYLOAD=$training_payload

env: TRAINING_PAYLOAD={
    "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
    "training_data_references": [
        {
            "type": "data_asset",
            "id": "credit_risk_training_light.csv",
            "connection": {},
            "location": {
                "href": "/v2/assets/f5531b4b-d1e8-44e7-a464-67164a2b8a6e?space_id=2219f598-6194-4e9a-9287-0730c69b5f4a"
            }
        }
    ],
    "results_reference": {
        "type": "fs",
        "id": "autoai_results",
        "connection": {},
        "location": {
            "path": "/spaces/2219f598-6194-4e9a-9287-0730c69b5f4a/assets/auto_ml/auto_ml_curl.eab90a7e0841c00db04266373c27ef71a0f312b5/wml_data"
        }
    },
    "tags": [
        {
            "value": "autoai"
        }
    ],
    "pipeline": {
        "href": "/v4/pipelines/a8cbf66e-2e7d-4ec4-b81c-0c24b38ef35f"
    }
}


In [58]:
%%bash --out training_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$TRAINING_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/trainings?version=2020-08-01" \
    | awk -F'"id":' '{print $2}' | cut -c2-37

In [59]:
%env TRAINING_ID=$training_id

env: TRAINING_ID=b4a501d9-8da3-47fa-9a2b-197d73fe84fb


<a id="training_details"></a>
### Get training details
Training is an asynchronous endpoint. In case you want to monitor training status and details,
you need to use a GET method and specify which training you want to monitor by usage of training ID.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_get" 
target="_blank" rel="noopener no referrer">Get information about training job</a>

### Get training status

In [61]:
%%bash

STATUS=$(curl -sk -X GET\
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/trainings/$TRAINING_ID?space_id=$SPACE_ID&version=2020-08-01")
    
STATUS=${STATUS#*state\":\"}
STATUS=${STATUS%%\"*}
echo $STATUS

completed


Please make sure that training is completed before you go to the next sections.
Monitor `state` of your training by running above cell couple of times.

<a id="runs"></a>
## 4. Historical runs

In this section you will see cURL examples describing how to get historical training runs information.

Output should be similar to the output from training creation but you should see more trainings entries.  
Listing trainings:

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_list" 
target="_blank" rel="noopener no referrer">Get list of historical training jobs information</a>

In [62]:
%%bash

HISTORICAL_TRAINING_LIMIT_TO_GET=2

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/trainings?space_id=$SPACE_ID&version=2020-08-01&limit=$HISTORICAL_TRAINING_LIMIT_TO_GET" \
    | python -m json.tool

{
    "limit": 2,
    "next": {
        "href": "/ml/v4/trainings?start=g1AAAAB4eJzLYWBgYMpgSmHgKy5JLCrJTq2MT8lPzkzJBYqrWCYnGxulmiXrpqampeqapBqk6lqaJZnopiWZJhqnmScZGKcagfRywPQSrSsLAN8IIc0&limit=2&space_id=2219f598-6194-4e9a-9287-0730c69b5f4a"
    },
    "resources": [
        {
            "metadata": {
                "created_at": "2020-10-01T06:50:05.311Z",
                "guid": "6e1bee43-062a-4d2b-9eeb-d4afd841a8ba",
                "id": "6e1bee43-062a-4d2b-9eeb-d4afd841a8ba",
                "modified_at": "2020-10-01T06:56:07.646Z",
                "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
                "tags": [
                    "autoai"
                ]
            },
            "entity": {
                "pipeline": {
                    "href": "/v4/pipelines/a8cbf66e-2e7d-4ec4-b81c-0c24b38ef35f",
                    "id": "a8cbf66e-2e7d-4ec4-b81c-0c24b38ef35f"
                },
                "results_reference": {
                    "location": {
   

<a id="training_cancel"></a>
### Cancel training run

**Tip:** If you want to cancel your training, please convert below cell to `code`, specify training ID and run.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_delete" 
target="_blank" rel="noopener no referrer">Canceling training</a>

---

<a id="deploy_and_score"></a>
## 5. Deploy and Score

In this section you will learn how to deploy and score pipeline model as webservice using WML instance.

Before deployment creation, you need store your model in WML repository.
Please see below cURL call example how to do it. Remember that you need to
specify where your chosen model is stored in COS.

<a id="model_store"></a>
### Store AutoAI model

Store information about your model to WML repository.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_create" 
target="_blank" rel="noopener no referrer">Model storing</a>

In [69]:
%%bash --out model_payload

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/v2/asset_files/auto_ml/auto_ml_curl.$RANDOM_ID/wml_data/$TRAINING_ID/assets/$TRAINING_ID""_P1_global_output/resources/wml_model/request.json?space_id=$SPACE_ID" \
    | python -m json.tool

In [71]:
%env MODEL_PAYLOAD=$model_payload

env: MODEL_PAYLOAD={
    "content_location": {
        "connection": {},
        "contents": [
            {
                "content_format": "native",
                "file_name": "pipeline_model.json",
                "location": "/spaces/2219f598-6194-4e9a-9287-0730c69b5f4a/assets/auto_ml/auto_ml_curl.eab90a7e0841c00db04266373c27ef71a0f312b5/wml_data/b4a501d9-8da3-47fa-9a2b-197d73fe84fb/assets/b4a501d9-8da3-47fa-9a2b-197d73fe84fb_P1_global_output/resources/wml_model/pipeline_model.json"
            },
            {
                "content_format": "pipeline-node",
                "file_name": "P1_automl.zip",
                "location": "/spaces/2219f598-6194-4e9a-9287-0730c69b5f4a/assets/auto_ml/auto_ml_curl.eab90a7e0841c00db04266373c27ef71a0f312b5/wml_data/b4a501d9-8da3-47fa-9a2b-197d73fe84fb/assets/b4a501d9-8da3-47fa-9a2b-197d73fe84fb_P1_global_output/resources/wml_model/P1_automl.zip",
                "pipeline_node_id": "automl"
            }
        ],
        "location": {


In [162]:
%%bash --out model_details

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$MODEL_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/models?version=2020-08-01&space_id=$SPACE_ID"

In [163]:
%env MODEL_DETAILS=$model_details

env: MODEL_DETAILS={
  "entity": {
    "content_import_state": "running",
    "hybrid_pipeline_software_specs": [{
      "id": "632d4b22-10aa-5180-88f0-f52dfb6444d7",
      "name": "autoai-kb_3.1-py3.7"
    }],
    "pipeline": {
      "id": "a8cbf66e-2e7d-4ec4-b81c-0c24b38ef35f"
    },
    "schemas": {
      "input": [{
        "fields": [{
          "name": "CheckingStatus",
          "type": "other"
        }, {
          "name": "LoanDuration",
          "type": "integer"
        }, {
          "name": "CreditHistory",
          "type": "other"
        }, {
          "name": "LoanPurpose",
          "type": "other"
        }, {
          "name": "LoanAmount",
          "type": "integer"
        }, {
          "name": "ExistingSavings",
          "type": "other"
        }, {
          "name": "EmploymentDuration",
          "type": "other"
        }, {
          "name": "InstallmentPercent",
          "type": "integer"
        }, {
          "name": "Sex",
          "type": "other"
 

In [164]:
%%bash --out model_id

echo $MODEL_DETAILS | awk -F '"id": ' '{ print $7 }' | cut -d '"' -f 2

In [165]:
%env MODEL_ID=$model_id

env: MODEL_ID=3f6336a3-b20d-4fd1-9ec6-07797344c799


<a id="model_content_download"></a>
### Download model content

If you want to download your saved model, please make the following call.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_filtered_download" 
target="_blank" rel="noopener no referrer">Download model content</a>

In [167]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --output "model.tar.gz" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/download?space_id=$SPACE_ID&version=2020-08-01"

In [168]:
!ls -l model.tar.gz

-rw-r--r--  1 amadeusz.masny1@ibm.com  staff  272 Oct  1 11:35 model.tar.gz


## <a id="deployment_creation"></a>
### Deployment creation

An AutoAI Batch deployment creation is presented below.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_create" 
target="_blank" rel="noopener no referrer">Create deployment</a>

In [169]:
%%bash --out deployment_payload

DEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "AutoAI deployment","description": "This is description","batch": {}, "hybrid_pipeline_hardware_specs": [{"node_runtime_id": "auto_ai.kb", "hardware_spec": {"name": "M"}}],"asset": {"id": "'"$MODEL_ID"'"}}'
echo $DEPLOYMENT_PAYLOAD | python -m json.tool

In [170]:
%env DEPLOYMENT_PAYLOAD=$deployment_payload

env: DEPLOYMENT_PAYLOAD={
    "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
    "name": "AutoAI deployment",
    "description": "This is description",
    "batch": {},
    "hybrid_pipeline_hardware_specs": [
        {
            "node_runtime_id": "auto_ai.kb",
            "hardware_spec": {
                "name": "M"
            }
        }
    ],
    "asset": {
        "id": "3f6336a3-b20d-4fd1-9ec6-07797344c799"
    }
}


In [177]:
%%bash --out deployment_details

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$DEPLOYMENT_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments?version=2020-08-01"

In [179]:
%env DEPLOYMENT_DETAILS=$deployment_details

env: DEPLOYMENT_DETAILS={
  "entity": {
    "asset": {
      "id": "3f6336a3-b20d-4fd1-9ec6-07797344c799"
    },
    "batch": {

    },
    "custom": {

    },
    "deployed_asset_type": "model",
    "description": "This is description",
    "hybrid_pipeline_hardware_specs": [{
      "hardware_spec": {
        "name": "M"
      },
      "node_runtime_id": "auto_ai.kb"
    }],
    "name": "AutoAI deployment",
    "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
    "status": {
      "state": "ready"
    }
  },
  "metadata": {
    "created_at": "2020-10-01T09:37:37.176Z",
    "description": "This is description",
    "id": "aa2cf720-a21f-4fa9-80d4-a4d1f845a7ee",
    "modified_at": "2020-10-01T09:37:37.176Z",
    "name": "AutoAI deployment",
    "owner": "1000330999",
    "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a"
  }
}


In [188]:
%%bash --out deployment_id

echo $DEPLOYMENT_DETAILS | awk -F '"id": ' '{ print $3 }' | cut -d '"' -f 2

In [189]:
%env DEPLOYMENT_ID=$deployment_id

env: DEPLOYMENT_ID=aa2cf720-a21f-4fa9-80d4-a4d1f845a7ee


<a id="deployment_details"></a>
### Get deployment details
As deployment API is asynchronous, please make sure your deployment is in `ready` state before going to the next points.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_get" 
target="_blank" rel="noopener no referrer">Get deployment details</a>

In [190]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "asset": {
            "id": "3f6336a3-b20d-4fd1-9ec6-07797344c799"
        },
        "batch": {},
        "custom": {},
        "deployed_asset_type": "model",
        "description": "This is description",
        "hybrid_pipeline_hardware_specs": [
            {
                "hardware_spec": {
                    "name": "M"
                },
                "node_runtime_id": "auto_ai.kb"
            }
        ],
        "name": "AutoAI deployment",
        "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
        "status": {
            "state": "ready"
        }
    },
    "metadata": {
        "created_at": "2020-10-01T09:37:37.176Z",
        "description": "This is description",
        "id": "aa2cf720-a21f-4fa9-80d4-a4d1f845a7ee",
        "modified_at": "2020-10-01T09:37:37.176Z",
        "name": "AutoAI deployment",
        "owner": "1000330999",
        "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a"
    }
}


<a id="batch_score"></a>
### Score your Batch deployment
Scoring for Batch deployment is done by creating `jobs`. User can specify job payload as a json or as data connection to eg. COS.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployment%20Jobs/deployment_jobs_create" 
target="_blank" rel="noopener no referrer">Create deployment job</a>

In [191]:
%%bash --out job_payload

JOB_PAYLOAD='{"name": "AutoAI job", "space_id": "'"$SPACE_ID"'","deployment": {"id": "'"$DEPLOYMENT_ID"'"}, "hybrid_pipeline_hardware_specs": [{"node_runtime_id": "auto_ai.kb", "hardware_spec": {"name": "M"}}], "scoring": {"input_data": [{"fields": ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings", "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration", "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents", "Telephone", "ForeignWorker"], "values": [["less_0", 6, "all_credits_paid_back", "car_used", 250, "less_100", "1_to_4", 2, "male", "none", 2, "savings_insurance", 28, "stores", "rent", 1, "skilled", 1, "none", "yes"]]}]}}'
echo $JOB_PAYLOAD | python -m json.tool

In [192]:
%env JOB_PAYLOAD=$job_payload

env: JOB_PAYLOAD={
    "name": "AutoAI job",
    "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
    "deployment": {
        "id": "aa2cf720-a21f-4fa9-80d4-a4d1f845a7ee"
    },
    "hybrid_pipeline_hardware_specs": [
        {
            "node_runtime_id": "auto_ai.kb",
            "hardware_spec": {
                "name": "M"
            }
        }
    ],
    "scoring": {
        "input_data": [
            {
                "fields": [
                    "CheckingStatus",
                    "LoanDuration",
                    "CreditHistory",
                    "LoanPurpose",
                    "LoanAmount",
                    "ExistingSavings",
                    "EmploymentDuration",
                    "InstallmentPercent",
                    "Sex",
                    "OthersOnLoan",
                    "CurrentResidenceDuration",
                    "OwnsProperty",
                    "Age",
                    "InstallmentPlans",
                    "Housing",
   

In [193]:
%%bash --out job_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$JOB_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployment_jobs?version=2020-08-01" \
    | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [194]:
%env JOB_ID=$job_id

env: JOB_ID=1ec9d794-0a12-4632-a9cd-4e5a2b842dcf


<a id="job_list"></a>
### Listing all Batch jobs

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployment%20Jobs/deployment_jobs_list" 
target="_blank" rel="noopener no referrer">List jobs</a>

In [196]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployment_jobs?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "resources": [
        {
            "entity": {
                "deployment": {
                    "id": "aa2cf720-a21f-4fa9-80d4-a4d1f845a7ee"
                },
                "hybrid_pipeline_hardware_specs": [
                    {
                        "hardware_spec": {
                            "name": "M"
                        },
                        "node_runtime_id": "auto_ai.kb"
                    }
                ],
                "platform_job": {
                    "job_id": "1a9a3796-3a77-43d8-9fe7-8a92f7199166",
                    "run_id": "b10a72d5-30d2-43b9-9694-7a2a928b8190"
                },
                "scoring": {
                    "input_data": [
                        {
                            "fields": [
                                "CheckingStatus",
                                "LoanDuration",
                                "CreditHistory",
                                "LoanPurpose",
                               

<a id="job_get"></a>
### Get particular job details

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployment%20Jobs/deployment_jobs_get" 
target="_blank" rel="noopener no referrer">Get job details</a>

In [197]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployment_jobs/$JOB_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "deployment": {
            "id": "aa2cf720-a21f-4fa9-80d4-a4d1f845a7ee"
        },
        "hybrid_pipeline_hardware_specs": [
            {
                "hardware_spec": {
                    "name": "M"
                },
                "node_runtime_id": "auto_ai.kb"
            }
        ],
        "platform_job": {
            "job_id": "1a9a3796-3a77-43d8-9fe7-8a92f7199166",
            "run_id": "b10a72d5-30d2-43b9-9694-7a2a928b8190"
        },
        "scoring": {
            "input_data": [
                {
                    "fields": [
                        "CheckingStatus",
                        "LoanDuration",
                        "CreditHistory",
                        "LoanPurpose",
                        "LoanAmount",
                        "ExistingSavings",
                        "EmploymentDuration",
                        "InstallmentPercent",
                        "Sex",
                        "OthersOnLoan",
        

<a id="job_cancel"></a>
### Cancel job

**Tip:** You can cancel running job by calling delete method.
Just convert below cell to `code` and run it.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployment%20Jobs/deployment_jobs_delete" 
target="_blank" rel="noopener no referrer">Cancel job</a>

<a id="deployments_list"></a>
### Listing all deployments

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_list" 
target="_blank" rel="noopener no referrer">List deployments details</a>

In [198]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployments?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "resources": [
        {
            "entity": {
                "asset": {
                    "id": "3f6336a3-b20d-4fd1-9ec6-07797344c799"
                },
                "batch": {},
                "custom": {},
                "deployed_asset_type": "model",
                "description": "This is description",
                "hybrid_pipeline_hardware_specs": [
                    {
                        "hardware_spec": {
                            "name": "M"
                        },
                        "node_runtime_id": "auto_ai.kb"
                    }
                ],
                "name": "AutoAI deployment",
                "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
                "status": {
                    "state": "ready"
                }
            },
            "metadata": {
                "created_at": "2020-10-01T09:35:43.215Z",
                "description": "This is description",
                "id": "4f3ba600-bbe4-4b13-a

<a id="cleaning"></a>
## 6. Cleaning section

Below section is useful when you want to clean all of your previous work within this notebook.
Just convert below cells into the `code` and run them.

<a id="training_delete"></a>
### Delete training run
**Tip:** You can completely delete a training run with its metadata.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_delete" 
target="_blank" rel="noopener no referrer">Deleting training</a>

<a id="job_delete"></a>
### Delete job

**Tip:** If you want remove job completely (with metadata), just specify `hard_delete` to True.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployment%20Jobs/deployment_jobs_delete" 
target="_blank" rel="noopener no referrer">Delete job</a>

<a id="deployment_delete"></a>
### Deleting deployment
**Tip:** You can delete existing deployment by calling DELETE method.

<a id="model_delete"></a>
### Delete model from repository
**Tip:** If you want to completely remove your stored model and model metadata, just use a DELETE method.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_delete" 
target="_blank" rel="noopener no referrer">Delete model from repository</a>

<a id="summary"></a>
## 7. Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use `cURL` calls to store, deploy and score a AutoAI model in WML. 


### Authors

**Amadeusz Masny**, Python Software Developer in Watson Machine Learning at IBM  
**Jan Sołtysik**, Intern in Watson Machine Learning at IBM

Copyright © 2020, 2021 IBM. This notebook and its source code are released under the terms of the MIT License.